In [1]:
# make sure ES is up and running
import requests
res = requests.get('http://localhost:9200')
print(res.content)

{
  "name" : "Sphinxor",
  "cluster_name" : "elasticsearch",
  "version" : {
    "number" : "2.4.0",
    "build_hash" : "ce9f0c7394dee074091dd1bc4e9469251181fc55",
    "build_timestamp" : "2016-08-29T09:14:17Z",
    "build_snapshot" : false,
    "lucene_version" : "5.5.2"
  },
  "tagline" : "You Know, for Search"
}



In [2]:
#connect to our cluster
from elasticsearch import Elasticsearch
es = Elasticsearch([{'host': 'localhost', 'port': 9200}])

In [3]:
#index some test data
es.index(index='test-index', doc_type='test', id=1, body={'test': 'test'})

{u'_id': u'1',
 u'_index': u'test-index',
 u'_shards': {u'failed': 0, u'successful': 1, u'total': 2},
 u'_type': u'test',
 u'_version': 1,
 u'created': True}

In [4]:
#delete test data and try with something more interesting
es.delete(index='test-index', doc_type='test', id=1)


{u'_id': u'1',
 u'_index': u'test-index',
 u'_shards': {u'failed': 0, u'successful': 1, u'total': 2},
 u'_type': u'test',
 u'_version': 2,
 u'found': True}

In [5]:
es.index(index='sw', doc_type='people', id=1, body={
	"name": "Luke Skywalker",
	"height": "172",
	"mass": "77",
	"hair_color": "blond",
	"birth_year": "19BBY",
	"gender": "male",
})

{u'_id': u'1',
 u'_index': u'sw',
 u'_shards': {u'failed': 0, u'successful': 1, u'total': 2},
 u'_type': u'people',
 u'_version': 1,
 u'created': True}

In [6]:
#let's iterate over swapi people documents and index them
import json
r = requests.get('http://localhost:9200') 
i = 1
while r.status_code == 200:
    r = requests.get('http://swapi.co/api/people/'+ str(i))
    es.index(index='sw', doc_type='people', id=i, body=json.loads(r.content))
    i=i+1
    
print(i)

18


In [7]:
#let's add more data but using node 2! Starting where the other stopped
import json

r = requests.get('http://localhost:9200') 
i = 18
while r.status_code == 200:
    r = requests.get('http://swapi.co/api/people/'+ str(i))
    es.index(index='sw', doc_type='people', id=i, body=json.loads(r.content))
    i=i+1
print(i)

90


In [8]:
es.get(index='sw', doc_type='people', id=65)

{u'_id': u'65',
 u'_index': u'sw',
 u'_source': {u'birth_year': u'40BBY',
  u'created': u'2014-12-20T16:46:40.440000Z',
  u'edited': u'2014-12-20T21:17:50.457000Z',
  u'eye_color': u'blue',
  u'films': [u'http://swapi.co/api/films/5/'],
  u'gender': u'female',
  u'hair_color': u'black',
  u'height': u'166',
  u'homeworld': u'http://swapi.co/api/planets/51/',
  u'mass': u'50',
  u'name': u'Barriss Offee',
  u'skin_color': u'yellow',
  u'species': [u'http://swapi.co/api/species/29/'],
  u'starships': [],
  u'url': u'http://swapi.co/api/people/65/',
  u'vehicles': []},
 u'_type': u'people',
 u'_version': 1,
 u'found': True}

In [9]:
#let's find darth vader
es.search(index="sw", body={"query": {"prefix" : { "name" : "Darth Vader" }}})

{u'_shards': {u'failed': 0, u'successful': 5, u'total': 5},
 u'hits': {u'hits': [], u'max_score': None, u'total': 0},
 u'timed_out': False,
 u'took': 116}

In [10]:
#let's see if we find anyone with a name or surname starting with lu
es.search(index="sw", body={"query": {"prefix" : { "name" : "lu" }}})

{u'_shards': {u'failed': 0, u'successful': 5, u'total': 5},
 u'hits': {u'hits': [{u'_id': u'1',
    u'_index': u'sw',
    u'_score': 1.0,
    u'_source': {u'birth_year': u'19BBY',
     u'created': u'2014-12-09T13:50:51.644000Z',
     u'edited': u'2014-12-20T21:17:56.891000Z',
     u'eye_color': u'blue',
     u'films': [u'http://swapi.co/api/films/6/',
      u'http://swapi.co/api/films/3/',
      u'http://swapi.co/api/films/2/',
      u'http://swapi.co/api/films/1/',
      u'http://swapi.co/api/films/7/'],
     u'gender': u'male',
     u'hair_color': u'blond',
     u'height': u'172',
     u'homeworld': u'http://swapi.co/api/planets/1/',
     u'mass': u'77',
     u'name': u'Luke Skywalker',
     u'skin_color': u'fair',
     u'species': [u'http://swapi.co/api/species/1/'],
     u'starships': [u'http://swapi.co/api/starships/12/',
      u'http://swapi.co/api/starships/22/'],
     u'url': u'http://swapi.co/api/people/1/',
     u'vehicles': [u'http://swapi.co/api/vehicles/14/',
      u'http:

In [11]:
#let's try a fuzzy query
q = es.search(index="sw", body={"query": {"fuzzy_like_this_field" : { "name" : {"like_text": "jaba", "max_query_terms":5}}}})

RequestError: TransportError(400, u'search_phase_execution_exception', u'No query registered for [fuzzy_like_this_field]')